# Dealing with imbalanced datasets, combining oversampling with VAE and undersampling to improve model recognition over all classes.  

In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.8 MB/s eta 0:00:00


Import packages, classifiers and etc.

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

from sklearn.metrics import confusion_matrix, make_scorer
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from collections import Counter

from imblearn.over_sampling import SMOTE, RandomOverSampler

Import VAEOversampler.

In [3]:
from VAEOversampler import VAEOversampler

## Loading data  
You can load some dataset from Imbalanced Learn list (https://imbalanced-learn.org/stable/datasets/index.html) or use your own data.  


In [4]:
from imblearn.datasets import fetch_datasets

dset_name = 'yeast_ml8'
dset = fetch_datasets()[dset_name]

X, y = StandardScaler().fit_transform(dset.data), dset.target

In [5]:
X

array([[ 0.9500571 ,  1.43265284,  0.64497763, ..., -0.433831  ,
         1.2690119 ,  1.1880271 ],
       [-0.24523899, -0.51160225, -0.36258548, ..., -0.13432431,
         0.24157372,  1.16943572],
       [-0.94033451,  0.22105826,  2.13831785, ..., -0.65652613,
        -0.90691512, -0.42282975],
       ...,
       [ 2.46867899,  1.30830839, -0.33578603, ..., -0.39466613,
        -0.2923849 , -0.01997965],
       [ 0.98675457,  0.9047637 ,  1.65112863, ..., -0.19583726,
         3.01240345,  1.36669356],
       [-0.02275421,  0.31605624,  0.07629516, ...,  0.08539816,
        -0.44657042, -1.54848887]])

In [6]:
y[y == -1] = 0

We split data into train and test partitions.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

This is a simple function to undersample freely.  

In [8]:
# RUS

def RUS(X_res, y_res, frac=1, minority_class_id=1, random_state=42):
    X_res = pd.DataFrame(X_res)
    X_res['Class'] = y_res

    X_neg = X_res[y_res != minority_class_id].sample(frac=frac, random_state=random_state)
    X_pos = X_res[y_res == minority_class_id].sample(frac=1, random_state=random_state)

    X_rus = pd.concat([X_neg, X_pos], ignore_index=True)

    X_eq = X_rus.drop('Class', axis=1)
    y_eq = X_rus['Class']

    return X_eq, y_eq

In [9]:
def train_val(X, y, Xt, yt, random_state=42):
    classifiers = {
        "CatBoostClassifier": CatBoostClassifier(verbose=False, random_seed=random_state),
        "LGBMClassifier": LGBMClassifier(random_state=random_state),
        "XGBClassifier": XGBClassifier(random_state=random_state),
        "BaggingClassifier": BaggingClassifier(random_state=random_state),
        "RandomForestClassifier": RandomForestClassifier(random_state=random_state),
    }
    scores = []
    predictions = []
    for key, classifier in classifiers.items():
        print('_' * 50)
        name = key
        classifier.fit(X, y)
        print("Classifier: ", name)
        y_pred = classifier.predict(Xt)
        cm = confusion_matrix(yt, y_pred)
        print(cm)
        print('')
        predictions.append(y_pred)
        tn = cm[0,0]
        fp = cm[0,1]
        fn = cm[1,0]
        tp = cm[1,1]
        tnr = tn / (tn + fp)
        tpr = tp / (tp + fn)
        scores.append(tnr * tpr)
        print('TNR:', round(tnr, 5))
        print('TPR:', round(tpr, 5))
        print('TNRxTPR:', round(tnr * tpr, 5))
        print('G-mean:', round(np.sqrt(tnr * tpr), 5))

    print('_' * 50)
    print('Ensemble predictions (majority voting):')
    predictions = np.sum(predictions, axis=0)
    predictions[predictions < 3] = 0
    predictions[predictions >= 3] = 1

    cm = confusion_matrix(yt, predictions)
    print(cm)
    tn = cm[0,0]
    fp = cm[0,1]
    fn = cm[1,0]
    tp = cm[1,1]
    tnr = tn / (tn + fp)
    tpr = tp / (tp + fn)
    print('')
    print('TNR:', round(tnr, 5))
    print('TPR:', round(tpr, 5))
    print('TNRxTPR:', round(tnr * tpr, 5))
    print('G-mean:', round(np.sqrt(tnr * tpr), 5))


## Without resampling (base line)
Which is starting score?


In [10]:
print('Original dataset shape %s' % Counter(y))
print('Ratio->', round(Counter(y)[0]/Counter(y)[1], 1), ': 1')

Original dataset shape Counter({0: 2239, 1: 178})
Ratio-> 12.6 : 1


In [11]:
train_val(X_train, y_train, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[448   0]
 [ 36   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  LGBMClassifier
[[448   0]
 [ 36   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  XGBClassifier
[[448   0]
 [ 36   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  BaggingClassifier
[[443   5]
 [ 36   0]]

TNR: 0.98884
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  RandomForestClassifier
[[448   0]
 [ 36   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Ensemble predictions (majority voting):
[[448   0]
 [ 36   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0


## Ratio 1:1  
Let's see classifiers scores when dataset is balanced.  


In [12]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

In [13]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0: 1791, 1: 1791})
Ratio->  1 : 1.0


In [14]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[437  11]
 [ 36   0]]

TNR: 0.97545
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  LGBMClassifier
[[435  13]
 [ 36   0]]

TNR: 0.97098
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  XGBClassifier
[[437  11]
 [ 35   1]]

TNR: 0.97545
TPR: 0.02778
TNRxTPR: 0.0271
G-mean: 0.16461
__________________________________________________
Classifier:  BaggingClassifier
[[434  14]
 [ 36   0]]

TNR: 0.96875
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  RandomForestClassifier
[[445   3]
 [ 36   0]]

TNR: 0.9933
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Ensemble predictions (majority voting):
[[442   6]
 [ 36   0]]

TNR: 0.98661
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0


In [15]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [16]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0: 1791, 1: 1791})
Ratio->  1 : 1.0


In [17]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[448   0]
 [ 36   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  LGBMClassifier
[[448   0]
 [ 36   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  XGBClassifier
[[448   0]
 [ 36   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  BaggingClassifier
[[443   5]
 [ 35   1]]

TNR: 0.98884
TPR: 0.02778
TNRxTPR: 0.02747
G-mean: 0.16573
__________________________________________________
Classifier:  RandomForestClassifier
[[448   0]
 [ 36   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Ensemble predictions (majority voting):
[[448   0]
 [ 36   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0


In [18]:
# VAEOversampler

vae_sampler = VAEOversampler(epochs=500,
                              intermediate_dim=512,
                              batch_size=64,
                              random_state=42,
                              verbose=False)
X_res, y_res = vae_sampler.fit_resample(X_train, y_train)

52/52 [==============================] - 0s 883us/step


In [19]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0.0: 1791, 1.0: 1791})
Ratio->  1 : 1.0


In [20]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[448   0]
 [ 36   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  LGBMClassifier
[[448   0]
 [ 36   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  XGBClassifier
[[447   1]
 [ 36   0]]

TNR: 0.99777
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  BaggingClassifier
[[448   0]
 [ 35   1]]

TNR: 1.0
TPR: 0.02778
TNRxTPR: 0.02778
G-mean: 0.16667
__________________________________________________
Classifier:  RandomForestClassifier
[[448   0]
 [ 36   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Ensemble predictions (majority voting):
[[448   0]
 [ 36   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0


## Under/Oversampling combination  
Now we can tuning the number of instances for each class to optimize metric.  


In [21]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

In [22]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.15)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 1791, 0: 269})
Ratio->  1 : 6.7


In [23]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[202 246]
 [  7  29]]

TNR: 0.45089
TPR: 0.80556
TNRxTPR: 0.36322
G-mean: 0.60268
__________________________________________________
Classifier:  LGBMClassifier
[[247 201]
 [ 11  25]]

TNR: 0.55134
TPR: 0.69444
TNRxTPR: 0.38287
G-mean: 0.61877
__________________________________________________
Classifier:  XGBClassifier
[[235 213]
 [ 11  25]]

TNR: 0.52455
TPR: 0.69444
TNRxTPR: 0.36427
G-mean: 0.60355
__________________________________________________
Classifier:  BaggingClassifier
[[285 163]
 [ 21  15]]

TNR: 0.63616
TPR: 0.41667
TNRxTPR: 0.26507
G-mean: 0.51485
__________________________________________________
Classifier:  RandomForestClassifier
[[248 200]
 [ 16  20]]

TNR: 0.55357
TPR: 0.55556
TNRxTPR: 0.30754
G-mean: 0.55456
__________________________________________________
Ensemble predictions (majority voting):
[[234 214]
 [ 10  26]]

TNR: 0.52232
TPR: 0.72222
TNRxTPR: 0.37723
G-mean: 0.61419


In [24]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [25]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.15)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 1791, 0: 269})
Ratio->  1 : 6.7


In [26]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[227 221]
 [ 16  20]]

TNR: 0.5067
TPR: 0.55556
TNRxTPR: 0.2815
G-mean: 0.53056
__________________________________________________
Classifier:  LGBMClassifier
[[351  97]
 [ 18  18]]

TNR: 0.78348
TPR: 0.5
TNRxTPR: 0.39174
G-mean: 0.62589
__________________________________________________
Classifier:  XGBClassifier
[[317 131]
 [ 13  23]]

TNR: 0.70759
TPR: 0.63889
TNRxTPR: 0.45207
G-mean: 0.67236
__________________________________________________
Classifier:  BaggingClassifier
[[279 169]
 [ 22  14]]

TNR: 0.62277
TPR: 0.38889
TNRxTPR: 0.24219
G-mean: 0.49213
__________________________________________________
Classifier:  RandomForestClassifier
[[282 166]
 [ 22  14]]

TNR: 0.62946
TPR: 0.38889
TNRxTPR: 0.24479
G-mean: 0.49476
__________________________________________________
Ensemble predictions (majority voting):
[[281 167]
 [ 20  16]]

TNR: 0.62723
TPR: 0.44444
TNRxTPR: 0.27877
G-mean: 0.52799


In [40]:
# VAEOversampler

X_res, y_res = vae_sampler.resample(X_train, y_train, sampling_strategy=3)

155/155 [==============================] - 0s 1ms/step


In [41]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.1)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1.0: 5089, 0.0: 179})
Ratio->  1 : 28.4


In [42]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[305 143]
 [ 12  24]]

TNR: 0.6808
TPR: 0.66667
TNRxTPR: 0.45387
G-mean: 0.6737
__________________________________________________
Classifier:  LGBMClassifier
[[302 146]
 [ 18  18]]

TNR: 0.67411
TPR: 0.5
TNRxTPR: 0.33705
G-mean: 0.58056
__________________________________________________
Classifier:  XGBClassifier
[[281 167]
 [ 22  14]]

TNR: 0.62723
TPR: 0.38889
TNRxTPR: 0.24392
G-mean: 0.49389
__________________________________________________
Classifier:  BaggingClassifier
[[324 124]
 [ 21  15]]

TNR: 0.72321
TPR: 0.41667
TNRxTPR: 0.30134
G-mean: 0.54894
__________________________________________________
Classifier:  RandomForestClassifier
[[329 119]
 [ 22  14]]

TNR: 0.73438
TPR: 0.38889
TNRxTPR: 0.28559
G-mean: 0.53441
__________________________________________________
Ensemble predictions (majority voting):
[[325 123]
 [ 18  18]]

TNR: 0.72545
TPR: 0.5
TNRxTPR: 0.36272
G-mean: 0.60227


## References  

  - Classification with Imbalanced Datasets:  
    https://sci2s.ugr.es/imbalanced  
  - Computer Vision:  Models, Learning, and Inference (Simon J.D. Prince):  
    http://www.computervisionmodels.com/  
  - Oversampling with VAEs:  
    https://towardsdatascience.com/oversampling-with-vaes-e410887fe51  
